In [53]:
# Pseudo random number generation 
# Linear congruential generators 
  # are an easy way to generate random number in a sequence
  # can be used to generate random numbers from uniform distribution
# X(n+1) = (a*X(n) + c) mod m  generating next random number recursively where x_o is the seed or start point
  # Above generate random numbers between 0-> m-1;
    
# Example generate uniform random numbers[0,1] via [0,m-1]/m and write to file 
# n = number of random numbers to generate, a,c are variables, m is the maximum value (0->m-1), x_0 seed start value

# Code courtesy of and adopted from https://tonypoer.io/2016/03/23/experimenting-with-linear-congruential-generators-in-python/ 


def linear_congruent_generator(n, a, c, m, x_0):
    """
    generates n random numbers between 0->m-1 using params a,c, and seed/start value x_0; writes output to txt file
    """
    # counter for how many iterations we've run
    i = 0  
    curr_value = x_0
    
    # Open a file for output
    outputFile = open("./Monte_Carlo_Methods_Finance/lcg_output.txt", "w")

    #Perfom number of iterations requested by user
    while i < n:
        # Store value of each iteration
        curr_value = (a * curr_value + c) % m

        #Obtain each number in U[0,1) by dividing X_i by m
        writeValue = str(curr_value/m)

        # write to output file
        outputFile.write(writeValue + "\n")
        # print "num: " + " " + str(counter) +":: " + str(x_value)

        i = i+1

    outputFile.close()
    print("Successfully stored " + str(n) + " random numbers in file named: 'lcg_output.txt'.")           
    
# Initialize variables
x_0  = 123456789.0         
a    = 101427 
c    = 321                  
m    = 2**16 
n    = 20

linear_congruent_generator(n,a,c,m,x_0)

Successfully stored 20 random numbers in file named: 'lcg_output.txt'.


In [56]:
# Generate random number from uniform distribution using scipy
from scipy.stats import uniform

uniform_array = uniform.rvs(size = 20)
uniform_array 

array([0.70483048, 0.47807858, 0.25281114, 0.56699283, 0.07052942,
       0.14699387, 0.99551633, 0.07802811, 0.01038133, 0.64214187,
       0.81913111, 0.80741977, 0.69114991, 0.79781204, 0.71492318,
       0.81509524, 0.02507727, 0.43078555, 0.2587518 , 0.14434655])

In [81]:
# Generating random numbers from various distributions using the inverse transform methods 
# Theorem :If we have U ~[0,1] and for some continous CDF cumulative distribution function 
# if we set  X = F'(U) the inverse, then X is distributed the same CDF as F

# Above relies on us having an inverse function of the CDF of the distribution we want to sample from

# Example sampling 20 valued from exponential distribution CDF = 1- expo(lambda*x) with lambda = 1
  # Inverse of exponential setting U = 1-exp(X) : X = -ln(1-U)/lambda
import math
import numpy as np
f = np.vectorize(math.log) #normally math.log takes a sinlge scalar value we can vectorize it to take array/nparray
X = -1*f(1-uniform_array)
X

array([1.22020543, 0.65023825, 0.2914373 , 0.83700098, 0.07314013,
       0.15898854, 5.40731301, 0.08124054, 0.01043559, 1.02761866,
       1.70998288, 1.64724246, 1.17489925, 1.59855754, 1.25499659,
       1.68791441, 0.02539706, 0.56349802, 0.29941976, 0.15588983])

In [88]:
# Rejection method as an improvement more reliable method than the inverse generation method 
# We want to generate random sample from distribution X with distribution f so we use another Y with distribution g to ...
# by getting samples from an area where the two distributions overlap through the use of constant c 
# c > f/g we want to choose Y that allows us to choose the smallest possible value of c
# 1. Generate a value u from U~[0,1]
# 2. Generate value y from some distribution Y
# 3. Check if u < f(y)/g(y)*c
#    if true set X = Y
#    else reject this value of y and repeat again

# Example using rejection method to sample from the normal distribution 
# X ~ N(0,1) we use Y ~ exp(lambda)
import math 
import numpy as np 
from scipy.stats import uniform

size_samples = 100 # number of samples to generate from normal distribution
norm_array   = [None]*size_samples
lambda_exp   = 1
i            = 0
c            = math.sqrt(2*math.exp(lambda_exp) / math.pi)

while i < size_samples: 
    uniform_val_1 = uniform.rvs(size =1) # generate number from uniform (here we use inbuilt scipy)
    uniform_val_2 = uniform.rvs(size =1) 
    exp_val       = -1*math.log(uniform_val_2) / lambda_exp  #use of inverse method to generate random value exponential
    norm_val      =  2*math.exp(-exp_val**2/2)/math.sqrt(2*math.pi) # apply y from exponential distribution to g(y) and f(y)
    if uniform_val_1 < norm_val/c*exp_val:
        uniform_val_3 = uniform.rvs(size=1)
        if uniform_val_3 <0.5:
            norm_array[i] = exp_val
        else: 
            norm_array[i] = exp_val
            i = i+1
    else:
        continue

norm_array
    

# Generate 30 random sample from normal distribution N~(0,1) using inbuilt functions
from scipy.stats import norm
norm_array_in = norm.rvs(size = 30 )
norm_array_in

array([ 0.77107462, -2.11006109,  0.43060319,  0.04374765,  0.09468009,
       -1.46561099, -0.82748567,  1.83154916,  0.43511084,  0.4390382 ,
        0.50770199,  1.60117414,  1.12042219, -2.01096678,  0.34939262,
        0.8816256 ,  0.07692431, -0.76918695,  0.01414185, -1.47878886,
       -0.59544931, -1.9868036 , -0.56965708, -0.2951161 ,  0.9862945 ,
        1.47840959,  1.71294904,  1.14346417,  0.77968507, -0.83927884])

In [91]:
# Generating random samples from Multivariate Normal Distribution for applicaitons options, stcoks, financial assets
# Financial assets tend to have some correlations that need to be taken into account
# If we have X ~ MVN(0,covMatrix) 
# The covariance matrix is psotive semi-definite so there exits some L such that LL' = covMatrix L' is the transpose of L
# X = LY where Y ~ MVN(0,I) I is the identity matrix
# To sample from correlated Multivariate Normal first sampele from uncorrelated then find L ( Cholesky Decompostion)

# Example generating multivariate normal from 2X2 i.e 2 assets
Sigma = np.array([[1,0.5],[0.5,1]])
L     = np.linalg.cholesky(Sigma)  # finding that special L such that LL' = Sigma

uncorr_sample = norm.rvs(size = 2)  # Y which is ~MVN(0,I)
corr_norms    = np.matmul(L,uncorr_sample) # X = LY
corr_norms

array([-1.07405852,  0.45443231])